# `virtual`: CSV Demo

In [1]:
%reload_ext autoreload
%autoreload 2

import virtual

csv_filepath = 'example.csv'

### One-shot table compression

In [2]:
# Save in Parquet format.
virtual.to_format(csv_filepath, 'example_virtual.csv', model_types=['sparse-lr'], prefix='demo-debug/')

Running schema inference..
Drilling functions..
     Earnings Year  Regular Earnings  Overtime Earnings  Total Earnings
722           2020          53353.99            2598.24        55952.23
77            2020           6403.80               0.00         6403.80
877           2020          15061.67               0.00        15061.67
613           2020           2583.70               0.00         2583.70
903           2020          39549.48               0.00        39549.48
..             ...               ...                ...             ...
835           2020          17824.52               0.00        17824.52
192           2020          76032.78             235.71        76268.49
629           2020          52957.48            8044.19        61001.67
559           2020          15074.50               0.00        15074.50
684           2020          42466.97            1971.51        44438.48

[999 rows x 4 columns]

BBBBBBBBB >>>Earnings Year
[7, 8, 9]
['Earnings Year', 'Regular

### Compare to vanilla CSV

In [8]:
import os

table = os.path.splitext(os.path.basename(csv_filepath))[0]
print(f'[{table}] CSV: {os.path.getsize('example.csv') / 1_000_000} MB')
print(f'[{table}] Virtual: {os.path.getsize('example_virtual.csv') / 1_000_000} MB')

CSV: 0.122375 MB
Virtual: 0.112833 MB


### Exploring functions

In [4]:
# Drill the functions.
functions = virtual.train(csv_filepath, model_types=['sparse-lr'], prefix='./demo-debug/')

functions

Drilling functions..
     Earnings Year  Regular Earnings  Overtime Earnings  Total Earnings
722           2020          53353.99            2598.24        55952.23
77            2020           6403.80               0.00         6403.80
877           2020          15061.67               0.00        15061.67
613           2020           2583.70               0.00         2583.70
903           2020          39549.48               0.00        39549.48
..             ...               ...                ...             ...
835           2020          17824.52               0.00        17824.52
192           2020          76032.78             235.71        76268.49
629           2020          52957.48            8044.19        61001.67
559           2020          15074.50               0.00        15074.50
684           2020          42466.97            1971.51        44438.48

[999 rows x 4 columns]

BBBBBBBBB >>>Earnings Year
[7, 8, 9]
['Earnings Year', 'Regular Earnings', 'Overtime Earni

[{'target_index': 0,
  'target_name': 'Earnings Year',
  'target_stats': {'mean': np.float64(2019.6546546546547),
   'max': 2022.0,
   'min': 2010.0},
  'models': {'sparse-lr': {'mse': 1.8572998848441455,
    'intercept': np.float64(2019.553519802266),
    'coeffs': [{'col-index': 7,
      'col-name': 'Regular Earnings',
      'coeff': np.float64(3.205195696083787e-06)}]}}},
 {'target_index': 9,
  'target_name': 'Total Earnings',
  'target_stats': {'mean': np.float64(32326.906086086085),
   'max': 202038.13,
   'min': 0.78},
  'models': {'sparse-lr': {'mse': 1.277106715403009e-11,
    'intercept': np.float64(-1.4551915228366852e-11),
    'coeffs': [{'col-index': 7,
      'col-name': 'Regular Earnings',
      'coeff': np.float64(1.0000000000000004)},
     {'col-index': 8,
      'col-name': 'Overtime Earnings',
      'coeff': np.float64(1.0000000000000007)}]}}}]

### Compress with functions

In [5]:
# Use them for compression.
virtual.to_format(csv_filepath, 'example_virtual.csv', functions)

Running schema inference..
Earnings Year sparse-lr -25
Total Earnings sparse-lr 6923
Creating the virtual file..


### Query the vanilla CSV file

In [6]:
import duckdb

duckdb.sql('select avg("Total Earnings") from read_csv("example.csv")').df()

,"avg(""Total Earnings"")"
0,32326.906086


### Query the virtualized CSV file

In [7]:
virtual.query(
  'select avg("Total Earnings") from read_csv("example_virtual.csv")',
  engine = 'duckdb'
)

,"avg(""Total Earnings"")"
0,32326.906086
